In [131]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Input, Lambda, Dropout, LSTM, Embedding, concatenate, Flatten, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K

from sklearn.model_selection import train_test_split
from sklearn import metrics

import pandas as pd
import numpy as np
import math

from processing import get_class_weights, get_prior, merge4Vectors, read_data, prepare_input
from processing import multiclass_encode_labels, apply_standard_scaler, apply_cat_encoder, binary_encode_labels

# Load the trained models

In [133]:
simple_NN = load_model("models/normal_NN.h5")
simple_NN_particlecount = load_model("models/normal_NN_particlecount.h5")
binary_LSTM = load_model("models/LSTM_binary.h5")
multiclass_LSTM = load_model("models/LSTM_multiclass.h5")
multioutput_LSTM = load_model("models/LSTM_multioutput.h5")

# Manually set Prior 

In [134]:
PRIOR = {'4top':0.04,
         'ttbar': 0.02,
         'ttbarHiggs': 0.19,
         'ttbarW': 0.51,
         'ttbarZ': 0.24}

PRIOR = np.array(list(PRIOR.values()))

# Load the data to be predicted 

In [135]:
df = read_data(FILENAME='ExamData.csv')
EVENTS = df['event ID']

# Prepare Data

In [136]:
FEATURES = [f'4Vector{x}' for x in range(1,11,1)]
CAT_FEATURES = [f'obj{x}' for x in range(1,11,1)]
MET_FEATURES = ['MET', 'METphi']
df, cat_df = prepare_input(df, particle_count=False)
df = merge4Vectors(df)
X_4VEC = np.array([np.vstack(col) for col in df[FEATURES].values])
X_CAT = cat_df[CAT_FEATURES].to_numpy()
X_MET = df[MET_FEATURES].to_numpy()

# Predict with binary model

In [143]:
binary_predictions = multioutput_LSTM.predict([X_4VEC, X_CAT, X_MET], verbose=1)
binary_predictions = binary_predictions[0] # discard multiclass output
print(binary_predictions.shape)

1563/1563 [==============================] - 33s 21ms/step
(50000, 1)


# Predict with MultiClass model

In [139]:
multiclass_predictions = multiclass_LSTM.predict([X_4VEC, X_CAT, X_MET], verbose=1)
print(multiclass_predictions.shape)

1563/1563 [==============================] - 36s 22ms/step
(50000, 5)


# Apply Prior

In [140]:
posterior = np.array([np.array(vect*(PRIOR)) for vect in multiclass_predictions])
# normalized_posterior = tf.keras.activations.softmax(tf.convert_to_tensor(posterior), axis=-1).numpy()

# Save predictions

In [144]:
binary_csv = pd.DataFrame(binary_predictions.T.tolist()[0], index=EVENTS)
binary_csv.to_csv('predictions/binary_predictions.csv', header=False)

In [145]:
multiclass_csv = pd.DataFrame(normalized_posterior.tolist(), index=EVENTS)
multiclass_csv.to_csv('predictions/multiclass_predictions.csv', header=False)